# Judea Pearl: The Ladder of Causation

### Loading Libraries

In [1]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# IterTools
from itertools import combinations

# Statistical Models
from scipy import stats

# Networkx
import networkx as nx

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

#### Setting Viz Parameters

In [2]:
plt.style.use('fivethirtyeight')

In [3]:
COLORS = [
    '#00B0F0',
    '#FF0000'
]

### Association

#### The Bookstore Example